## 定数の設定

In [ ]:
## FQ データが格納された SQLite DB の URL を以下に指定します
DB_URL = ""

## 抽出結果を保存したいスプレッドシートの URL を以下に指定します
SHEET_URL = ""

## 実行したい SQL クエリを以下に記述します
SQL_QUERY = """

"""

## 証券コードなどで抽出対象を絞りたい場合は、上記のクエリとは別に、
## 当該証券コードのリストを以下に指定します
## 詳細は「抽出対象企業群の絞り込み（希望する場合のみ）」セクションでおこなうので、
## ここは None のままでかまいません
FILTER_LIST = None


## リポジトリの読み込み

In [ ]:
import sys
import os

# 読み込みたいリポジトリのリスト (URL, ディレクトリ名)
REPOSITORIES = [
    ("https://github.com/ugohsu/colab-fqdata.git", "colab-fqdata"),
    ("https://github.com/ugohsu/colab-common.git", "colab-common"),
]

for repo_url, dir_name in REPOSITORIES:
    repo_path = f"/content/{dir_name}"

    # ディレクトリが存在するか確認
    if not os.path.exists(repo_path):
        # 存在しない場合は clone
        print(f"Cloning {dir_name}...")
        !git clone $repo_url
    else:
        # 存在する場合は pull (更新)
        print(f"Updating {dir_name}...")
        !git -C $repo_path pull

    # パスを追加 (重複追加を防ぐため、未登録の場合のみ追加)
    if repo_path not in sys.path:
        sys.path.append(repo_path)

print("Setup completed.")

Cloning colab-fqdata...
Cloning into 'colab-fqdata'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 6), reused 30 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 5.64 KiB | 824.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.
Cloning colab-common...
Cloning into 'colab-common'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 187 (delta 104), reused 130 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (187/187), 50.21 KiB | 4.18 MiB/s, done.
Resolving deltas: 100% (104/104), done.
Setup completed.


## 抽出対象企業群の絞り込み（希望する場合のみ）

証券コードで絞り込みをおこないたい場合は、以下のコードをコードブロックにコピーし、適宜書き換えて、実行します。

```python
from colab_common import read_df_from_gsheet

## スプレッドシートの読み込み
df_temporary = read_df_from_gsheet(
    SHEET_URL,
    sheet_name="証券コードリスト"  # 自身の環境にあわせてシート名を書き換えます
)

## 証券コードリストが含まれる列の読み込み
FILTER_LIST = df_temporary["証券コード"] # 自身の環境にあわせて列名を書き換えます
```

## FQ データの抽出

In [ ]:
from colab_fqdata import FqLoader
loader = FqLoader(DB_URL)
loaded_data = loader.read_sql(SQL_QUERY, filter_list = FILTER_LIST)

## 抽出データの書き込み

In [ ]:
from colab_common import write_df_to_gsheet
write_df_to_gsheet(loaded_data, SHEET_URL)